In [4]:
# Imports
import twint
import numpy as np
import pandas as pd
import datetime
import schedule
import time
import os
import glob
from IPython.display import clear_output
import nest_asyncio
nest_asyncio.apply()

In [1]:
# Keywordliste einlesen
def readKeywordFile(file):
    keywords = []
    with open(file, "r") as filestream:
        for i, line in enumerate(filestream):
            currentline = line.split(",")
            for eintrag in currentline:
                keywords.append(eintrag.replace(" ", "").replace("\n", "").replace("ae", "ä").replace("oe", "ö").replace("ue", "ü"))
    return list(filter(None, keywords))

# Twitter durchsuchen nach Keyword mit Parametern
def searchTwitterTwint(keyword, since=None, until=None, language=None, limit=None, outputFile=False, outputFileName="", outputFolder="", createDataFrame=True, hide_out=True):
    c = twint.Config()
    c.Search = keyword
    c.Filter_retweets = False
    if language is not None:
        c.Lang = language
    if since is not None:
        c.Since = since
    if until is not None:
        c.Until = until
    if limit is not None:
        c.Limit = limit
    c.Hide_output = hide_out
    if outputFile:
        c.Store_csv = True
        if outputFile == "":
            outputFile = "Default"
        if outputFolder == "":
            outputFolder = "data"
        #c.Output = "{}.csv".format(outputFileName)
        c.Output = os.path.join('../Daten/TwintApi/{}/{}.csv'.format(outputFolder, outputFileName))
    if createDataFrame:
        c.Pandas = True
    twint.run.Search(c)
    if createDataFrame:
        Tweets_df = twint.storage.panda.Tweets_df
        return Tweets_df
        
# Für den vorherigen Tag alle Tweets der Keywordliste farmen    
def job_EveryDay(keywords):
    schedule.clear()
    clear_output(wait=True)
    today = datetime.datetime.now().strftime("%Y-%m-%d")
    yesterday = (datetime.datetime.now() - pd.to_timedelta(1,'d')).strftime("%Y-%m-%d")
    if not os.path.exists("../Daten/TwintApi/{}".format(yesterday)):
        os.mkdir("../Daten/TwintApi/{}".format(yesterday))
    for i, keyword in enumerate(keywords):
        clear_output(wait=True)
        print("Akutelles Keyword: ", keyword)
        #filename = "{}_{}".format(yesterday, i) # Nummer
        filename = "{}_{}".format(yesterday, keyword)  # Keyword
        searchTwitterTwint(createDataFrame=False, language="de", keyword="{} lang:de".format(keyword), since=yesterday, until=today, hide_out=True, outputFile=True, outputFileName=filename, outputFolder=yesterday)
        time.sleep(1)

# Alle Tweets der Keywordliste von einem Datum farmen        
def getTwitterFromDay(date, keywords):
    clear_output(wait=True)
    yesterday = datetime.datetime.strptime(date, "%Y-%m-%d").strftime("%Y-%m-%d")
    till = (datetime.datetime.strptime(date, "%Y-%m-%d") + pd.to_timedelta(1,'d')).strftime("%Y-%m-%d")
    #print("Daten zwischen: ", yesterday, " : ", till)
    if not os.path.exists("../Daten/TwintApi/{}".format(yesterday)):
        os.mkdir("../Daten/TwintApi/{}".format(yesterday))
    for i, keyword in enumerate(keywords):
        clear_output(wait=True)
        print("Daten zwischen: ", yesterday, " : ", till)
        print("Akutelles Keyword: ", keyword)
        #filename = "{}_{}".format(yesterday, i) # Nummer
        filename = "{}_{}".format(yesterday, keyword)  # Keyword
        searchTwitterTwint(createDataFrame=False, language="de", keyword="{} lang:de".format(keyword), since=yesterday, until=till, hide_out=True, outputFile=True, outputFileName=filename, outputFolder=yesterday)
        time.sleep(1)

# Alle Tweets der Keywordliste in einem Zeitbereich farmen       
def getTwitterBetweenDates(fromDate, tillDate, keywords):
    fromThisDate = datetime.datetime.strptime(fromDate, "%Y-%m-%d")
    toThisDate = datetime.datetime.strptime(tillDate, "%Y-%m-%d")
    listWithDates = pd.date_range(fromThisDate,(toThisDate-pd.to_timedelta(1,'d')),freq='d')
    #print(listWithDates[0].strftime("%Y-%m-%d"))
    for date in listWithDates:
        getTwitterFromDay(date.strftime("%Y-%m-%d"), keywords)
    print("Ende")

In [2]:
keywords = readKeywordFile("./KeywordsGER.txt")

In [5]:
job_EveryDay(keywords)

Akutelles Keyword:  #fdp
[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.


In [ ]:
schedule.every().day.at("12:00").do(job_EveryDay, keywords)
while True:
    schedule.run_pending()
    time.sleep(60) # Check every minute

In [1]:
#getTwitterFromDay("2021-03-10", keywords)

In [2]:
#getTwitterBetweenDates("2021-01-01", "2021-03-29", keywords[234:])